In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')
MODEL=os.getenv('LLM_MODEL')

In [2]:
# For OpenAI after 1.0.0 (1.13.3 at the time of writing)
client = openai.OpenAI()

def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

![](../images/06-iterative.png)

![](../images/07-process.png)

## Generate a marketing product description from a product fact sheet

In [3]:
fact_sheet_chair = """
OVERVIEW
- Part of a beautiful family of mid-century inspired office furniture, 
including filing cabinets, desks, bookcases, meeting tables, and more.
- Several options of shell color and base finishes.
- Available with plastic back and front upholstery (SWC-100) 
or full upholstery (SWC-110) in 10 fabric and 6 leather options.
- Base finish options are: stainless steel, matte black, 
gloss white, or chrome.
- Chair is available with or without armrests.
- Suitable for home or business settings.
- Qualified for contract use.

CONSTRUCTION
- 5-wheel plastic coated aluminum base.
- Pneumatic chair adjust for easy raise/lower action.

DIMENSIONS
- WIDTH 53 CM | 20.87”
- DEPTH 51 CM | 20.08”
- HEIGHT 80 CM | 31.50”
- SEAT HEIGHT 44 CM | 17.32”
- SEAT DEPTH 41 CM | 16.14”

OPTIONS
- Soft or hard-floor caster options.
- Two choices of seat foam densities: 
 medium (1.8 lb/ft3) or high (2.8 lb/ft3)
- Armless or 8 position PU armrests 

MATERIALS
SHELL BASE GLIDER
- Cast Aluminum with modified nylon PA6/PA66 coating.
- Shell thickness: 10 mm.
SEAT
- HD36 foam

COUNTRY OF ORIGIN
- Italy
"""

In [4]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

Introducing our mid-century inspired office chair, a perfect blend of style and functionality. This chair is part of an elegant collection of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. 

You can customize this chair to match your office decor with several shell color and base finish options. Choose from stainless steel, matte black, gloss white, or chrome for the base finish. The chair is available with a plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options. 

This chair is not only stylish but also highly functional. It features a 5-wheel plastic coated aluminum base for easy mobility and a pneumatic chair adjust for easy raise/lower action. The chair is available with or without armrests, and you can choose between soft or hard-floor caster options. 

The seat is made of HD36 foam and comes in two foam densities: medium (1.8 lb/ft3) or high (2.8 lb/ft3). The shell base glider is ma

## Issue 1: The text is too long 
- Limit the number of words/sentences/characters.

In [5]:
len(response)

1473

In [6]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

This mid-century inspired office chair features a 5-wheel plastic coated aluminum base and a shell made from cast aluminum with a modified nylon PA6/PA66 coating. It offers a choice of plastic or fully upholstered finishes, and optional armrests. Suitable for both soft and hard floors. Made in Italy.


Try to say "Use at most 3 sentences" or "Use at most 280 characters"

## Issue 2. Text focuses on the wrong details
- Ask it to focus on the aspects that are relevant to the intended audience.

In [7]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

This mid-century inspired office chair features a 5-wheel plastic coated aluminum base and a shell made from cast aluminum with a modified nylon PA6/PA66 coating. It offers a choice of plastic or fully upholstered finishes, with a seat made from HD36 foam. Available with or without armrests.


In [8]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

Use at most 50 words.

Technical specifications: ```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)

This mid-century inspired office chair is constructed from a cast aluminum shell base with a modified nylon PA6/PA66 coating, ensuring durability. The seat is made from HD36 foam for comfort. It features a 5-wheel plastic coated aluminum base for easy mobility. Product IDs: SWC-100, SWC-110.


## Issue 3. Description needs a table of dimensions
- Ask it to extract information and organize it in a table.

In [9]:
prompt = f"""
Your task is to help a marketing team create a 
description for a retail website of a product based 
on a technical fact sheet.

Write a product description based on the information 
provided in the technical specifications delimited by 
triple backticks.

The description is intended for furniture retailers, 
so should be technical in nature and focus on the 
materials the product is constructed from.

At the end of the description, include every 7-character 
Product ID in the technical specification.

After the description, include a table that gives the 
product's dimensions. The table should have two columns.
In the first column include the name of the dimension. 
In the second column include the measurements in inches only.

Give the table the title 'Product Dimensions'.

Format everything as HTML that can be used in a website. 
Place the description in a <div> element.

Technical specifications: ```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)

<div>
<p>This mid-century inspired office chair is part of a beautiful family of office furniture that includes filing cabinets, desks, bookcases, meeting tables, and more. The chair is available in several shell colors and base finishes, and can be ordered with or without armrests. It is suitable for both home and business settings and is qualified for contract use.</p>

<p>The chair's base is a 5-wheel plastic coated aluminum base, providing stability and easy movement. The pneumatic chair adjust allows for easy raise and lower action, ensuring comfort for users of all heights. The chair is available with either a plastic back and front upholstery (SWC-100) or full upholstery (SWC-110) in 10 fabric and 6 leather options. Base finish options include stainless steel, matte black, gloss white, or chrome.</p>

<p>The chair's shell base glider is made of cast aluminum with a modified nylon PA6/PA66 coating, ensuring durability and longevity. The shell thickness is 10 mm. The seat is made 

## Load Python libraries to view HTML

In [10]:
from IPython.display import display, HTML

In [11]:
display(HTML(response))

Dimension,Measurement (inches)
Width,20.87
Depth,20.08
Height,31.50
Seat Height,17.32
Seat Depth,16.14


## Try experimenting on your own!